In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
# Initialize the model once outside the loop for efficiency
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\victo\anaconda3\envs\workproject\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
job_embeddings = model.encode("python")

cand_emded = model.encode("rust")

joker = model.encode("crocodile")

In [24]:
print(cosine_similarity(job_embeddings.reshape(1,-1), cand_emded.reshape(1,-1)))
print(cosine_similarity(job_embeddings.reshape(1,-1), joker.reshape(1,-1)))

[[0.2863149]]
[[0.40933645]]


In [25]:
job_embeddings = model.encode("python,c++,julia,oracle")

cand_emded = model.encode("rust,go,sql,php")

joker = model.encode("crocodile,lizard,boa,rattlesnake")

In [26]:
print(cosine_similarity(job_embeddings.reshape(1,-1), cand_emded.reshape(1,-1)))
print(cosine_similarity(job_embeddings.reshape(1,-1), joker.reshape(1,-1)))

[[0.34695765]]
[[0.11074138]]


In [4]:
import gensim

In [5]:
from gensim.models import Word2Vec

# Example corpus for training
corpus = [
    ["Python", "is", "a", "popular", "programming", "language"],
    ["Java", "and", "C++", "are", "also", "programming", "languages"],
    ["Python", "can", "be", "used", "for", "machine", "learning"],
    ["Gensim", "is", "a", "library", "for", "natural", "language", "processing"],
]

# Train a Word2Vec model
model = Word2Vec(sentences=corpus, vector_size=50, window=3, min_count=1, workers=4)

# Find most similar words to "Python"
similar_words = model.wv.most_similar("Python", topn=5)

# Display the similar words
print("Most similar words to 'Python':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")


Most similar words to 'Python':
learning: 0.2707
can: 0.2106
processing: 0.1672
languages: 0.1503
popular: 0.1321


In [ ]:
import gensim.downloader as api

# Download the Google News vectors
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Word 'sus' is in the model.


In [7]:
if 'python' in model:
    similar_words = model.most_similar('python', topn=10)
    for word, similarity in similar_words:
        print(f"{word}: {similarity}")
else:
    print("Word 'python' is not in the vocabulary.")

pythons: 0.6688377857208252
Burmese_python: 0.6680364608764648
snake: 0.6606292724609375
crocodile: 0.6591362953186035
boa_constrictor: 0.6443519592285156
alligator: 0.6421656608581543
reptile: 0.6387744545936584
albino_python: 0.6158879995346069
croc: 0.6083583831787109
lizard: 0.6013416647911072


In [9]:
similar_words = model.most_similar('BERT', topn=10)

In [1]:
for word, similarity in similar_words:
        print(f"{word}: {similarity}")

NameError: name 'similar_words' is not defined

In [ ]:
%%bash
# Download the GloVe embeddings
wget http://nlp.stanford.edu/data/glove.6B.zip

# Extract the GloVe embeddings
unzip glove.6B.zip -d glove.6B

/usr/bin/bash: line 2: wget: command not found


Archive:  glove.6B.zip
  inflating: glove.6B/glove.6B.50d.txt  
  inflating: glove.6B/glove.6B.100d.txt  
  inflating: glove.6B/glove.6B.200d.txt  
  inflating: glove.6B/glove.6B.300d.txt  


In [10]:
import os
from gensim.models.keyedvectors import KeyedVectors

# Convert GloVe format to Gensim format
def load_glove_model(glove_file):
    glove_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)
    return glove_vectors

# Load the 300-dimensional GloVe vectors
glove_file = os.path.join("glove.6B", "glove.6B.300d.txt")
glove_model = load_glove_model(glove_file)


In [11]:
if 'python' in glove_model:
    similar_words = glove_model.most_similar('python', topn=10)
    print("Words similar to 'python':")
    for word, similarity in similar_words:
        print(f"{word}: {similarity}")
else:
    print("Word 'python' is not in the vocabulary.")


Words similar to 'python':
monty: 0.6837380528450012
perl: 0.519283652305603
cleese: 0.5092198252677917
pythons: 0.5007115006446838
php: 0.4942314028739929
grail: 0.4683017134666443
scripting: 0.46761268377304077
skit: 0.4474538266658783
javascript: 0.4312553107738495
spamalot: 0.43117913603782654


In [17]:
word = "kurtosis"

if word in glove_model:
    similar_words = glove_model.most_similar(word, topn=10)
    print(f"Words similar to {word}:")
    for this, similarity in similar_words:
        print(f"{this}: {similarity}")
else:
    print(f"Word {word} is not in the vocabulary.")

Words similar to kurtosis:
skewness: 0.6981668472290039
cumulant: 0.3952988386154175
formula_70: 0.3895203471183777
formula_69: 0.38466396927833557
alphas: 0.38312193751335144
multinomial: 0.3823775351047516
k-space: 0.37801066040992737
kt/v: 0.3779199421405792
vesca: 0.37692299485206604
infimum: 0.3755549192428589


In [21]:
import string

voc = string.ascii_letters

for v in voc:
    print(v, end="-")

a-b-c-d-e-f-g-h-i-j-k-l-m-n-o-p-q-r-s-t-u-v-w-x-y-z-A-B-C-D-E-F-G-H-I-J-K-L-M-N-O-P-Q-R-S-T-U-V-W-X-Y-Z-